# In this Notebook we are attempting to predict if a person will win their next fight depending on how they performed on their last 5 fights

## Library Imports

In [ ]:
import sys
sys.path.insert(1, '../combined_data')

import pandas as pd
import numpy as np
import talos as ta

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support

from talos.model.normalizers import lr_normalizer
from talos.model.hidden_layers import hidden_layers

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, Flatten, MaxPooling1D, LSTM
from keras.optimizers import Adam, Nadam, RMSprop
from keras.losses import logcosh, binary_crossentropy
from keras.activations import relu, elu, sigmoid

import matplotlib.pyplot as plt
%matplotlib inline

from make_career import make_career

## Loading the dataset

In [ ]:
features, labels = make_career(N_FIGHT_CAREER=5)
labels = labels.reshape(-1,)
score_cols = ['precision', 'recall', 'fbeta_score', 'support']

In [ ]:
print('Features {}'.format(features.shape))
print('Labels {}'.format(labels.shape))

## Preprocess Features

## Generates np.array of shape (3220, 5 * features)

In [ ]:
def collapse_n_fights():
    num_cols, cat_cols = get_column_types()
    
    # Collect only the features we want
    new_features = []
    for window in features:
        window_arr = list(window.reshape(1,-1))[0]
        fight_window = np.array([])
        for fight in window_arr:
            fight_arr = np.array([fight[i] for i in num_cols])
            np.nan_to_num(fight_arr, copy=False)
            fight_window = np.append(fight_window, fight_arr)
        new_features.append(fight_window)
    
    new_features, new_labels = shuffle(np.array(new_features), labels)
    X_train, X_test, y_train, y_test = train_test_split(new_features, new_labels, random_state=0, train_size=0.90)
    
    # Scale the data
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test

def plot_training_results(history_obj):
    acc = history_obj.history['accuracy']
    val_acc = history_obj.history['val_accuracy']
    loss = history_obj.history['loss']
    val_loss = history_obj.history['val_loss']
    
    epochs = range(1, len(acc) + 1)

    fig, (accuracy_axis, loss_axis) = plt.subplots(1, 2, figsize=(20, 5))

    accuracy_axis.plot(epochs, acc, 'bo', label='Training acc')
    accuracy_axis.plot(epochs, val_acc, 'b', label='Validation acc')
    accuracy_axis.set_title('Training and validation accuracy')
    accuracy_axis.set_ylabel('Accuracy')
    accuracy_axis.set_xlabel('Epoch')
    accuracy_axis.legend()

    loss_axis.plot(epochs, loss, 'bo', label='Training loss')
    loss_axis.plot(epochs, val_loss, 'b', label='Validation loss')
    loss_axis.set_title('Training and validation loss')
    loss_axis.set_ylim(0, 3)
    loss_axis.set_ylabel('Loss')
    loss_axis.set_xlabel('Epoch')
    loss_axis.legend()
    fig.show()

## Model Parameters and Definitions

In [ ]:
# Define hyperparameters to use in Grid Search for DNN
dnn_params = {
     'lr': [0.01, 0.1, 1],
     'first_neuron': [128],
     'hidden_layers': [1],
     'batch_size': [64],
     'epochs': [10],
     'dropout': [0.01],
     'optimizer': [Adam],
     'shapes':['funnel'],
     'losses': [binary_crossentropy],
     'activation': [relu],
     'last_activation': [sigmoid],
}

# dnn_params = {
#     'lr': [0.01, 0.1, 1],
#     'first_neuron': [128],
#     'hidden_layers': [1],
#     'batch_size': [64, 128, 256],
#     'epochs': [10, 25],
#     'dropout': [0.01, 0.1],
#     'optimizer': [Adam],
#     'shapes':['funnel'],
#     'losses': [binary_crossentropy],
#     'activation': [relu],
#     'last_activation': [sigmoid],
# }

# Define hyperparameters to use in Grid Search for CNN
# cnn_params = {
#     'lr': [0.01],
#     'num_filters': [64],
#     'kernel_size': [2],
#     'batch_size': [64],
#     'epochs': [5],
#     'dropout': [0.01],
#     'flatten_layer': [100], 
#     'optimizer': [Adam],
#     'losses': [binary_crossentropy],
#     'activation': [relu],
#     'last_activation': [sigmoid],
# }

# Longer Run
cnn_params = {
    'lr': [0.01, 0.1, 1],
    'num_filters': [64, 128],
    'kernel_size': [2],
    'batch_size': [64],
    'epochs': [5, 15],
    'dropout': [0.01],
    'flatten_layer': [100], 
    'optimizer': [Adam],
    'losses': [binary_crossentropy],
    'activation': [relu],
    'last_activation': [sigmoid],
}

In [ ]:
# Get the first fight window and the first fight in that window
def get_column_types():
    num_cols = []
    cat_cols = []
    for fight in features[0][0]:
        feature_type = type(features[1][1][fight])
        if feature_type is not float and feature_type is not int:
            cat_cols.append(fight)
        else:
            num_cols.append(fight)
            
    return num_cols, cat_cols

In [ ]:
dnn_scores = []
def get_dnn(x_train, y_train, x_val, y_val, params, test_model=False):
    
    model = Sequential()
    # Input Layer
    model.add(Dense(params["first_neuron"], 
                    activation=params['activation'], 
                    input_dim=X_train.shape[1]))
    
    model.add(Dropout(params['dropout']))
    
    # Hidden Layers
    hidden_layers(model, params, 1)
    
    # Output Layers
    model.add(Dense(1, activation=params['last_activation']))
    
    model.compile(loss=params['losses'],
                  optimizer=params['optimizer'](lr=lr_normalizer(params['lr'], params['optimizer'])), 
                  metrics=['accuracy'])
                  
    history = model.fit(x_train, y_train,
                       validation_data=[x_val, y_val],
                       batch_size=params['batch_size'],
                       epochs=params['epochs'],
                       verbose=0)
    
    score = precision_recall_fscore_support(y_val, model.predict_classes(x_val), average='binary')
    dnn_scores.append(score)
    
    if test_model:
        model.summary()
        plot_training_results(history)
        

    return history, model


cnn_scores = []
def get_cnn(x_train, y_train, x_val, y_val, params, test_model=False):
    model = Sequential()
    
    # Convolutional Layers
    model.add(Conv1D(filters=params["num_filters"], kernel_size=params["kernel_size"], activation='relu', input_shape=(5, 97)))
    model.add(Conv1D(filters=params["num_filters"], kernel_size=params["kernel_size"], activation='relu'))
    model.add(Dropout(params['dropout']))
    
    # Flatten Layers
    model.add(Flatten())
    model.add(Dense(params["flatten_layer"], activation='relu'))
    
    # Output Layer
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss=params['losses'],
                  optimizer=params['optimizer'](lr=lr_normalizer(params['lr'], params['optimizer'])), 
                  metrics=['accuracy'])
    
    history = model.fit(x_train, y_train,
                       validation_data=[x_val, y_val],
                       batch_size=params['batch_size'],
                       epochs=params['epochs'],
                       verbose=0)
    
    score = precision_recall_fscore_support(y_val, model.predict_classes(x_val), average='binary')
    cnn_scores.append(score)
    
    if test_model:
        plot_training_results(history)
        model.summary()

    return history, model


lstm_scores = []
def get_lstm(x_train, y_train, x_val, y_val, test_model=False):
    model = Sequential()
    
    model.add(LSTM(50, 
                   input_shape=(5, 97),
                   recurrent_dropout=0.2)) #, return_sequences=True
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss="binary_crossentropy", 
                  optimizer=Adam(learning_rate=0.01), 
                  metrics=['accuracy'])
    
    history = model.fit(x_train, y_train,
                       validation_data=[x_val, y_val],
                       batch_size=64,
                       epochs=30,
                       verbose=1)
    
    score = precision_recall_fscore_support(y_val, model.predict_classes(x_val), average='binary')
    lstm_scores.append(score)
    
    if test_model:
        plot_training_results(history)
        model.summary()

    return history, model

## Building a DNN to predict the winner using last n fights

In [ ]:
# Get the data
X_train, X_test, y_train, y_test = collapse_n_fights()

# Create the Neural Network
dnn_model = ta.Scan(
    x=X_train,
    y=y_train,
    model=get_dnn,
    params=dnn_params,
    experiment_name='LAST_5_DNN',
)

In [ ]:
results_df = dnn_model.data
dnn_cols = list(results_df.columns)

new_df_data = []
for index, row_data in results_df.iterrows():
    new_row = dict()
    
    for col in dnn_cols:
        new_row[col] = row_data[col]
    
    for score_index, col in enumerate(score_cols):
        new_row[col] = dnn_scores[index][score_index]
        
    new_df_data.append(new_row)

dnn_df = pd.DataFrame(new_df_data)
dnn_df.drop(columns=['round_epochs', 'batch_size', 'epochs', 'support', 'precision', 'recall', 'shapes', 'first_neuron'], inplace=True)
top_5_val_dnn = dnn_df.sort_values(by=['val_accuracy'], ascending=False).head(5)
top_5_val_dnn

## Building a CNN to predict the winner using last n fights

In [ ]:
# Get the data
X_train, X_test, y_train, y_test = collapse_n_fights()
X_train = X_train.reshape(X_train.shape[0], 5, 97)
X_test = X_test.reshape(X_test.shape[0], 5, 97)

# Create the Neural Network
cnn_model = ta.Scan(
    x=X_train,
    y=y_train,
    model=get_cnn,
    params=cnn_params,
    experiment_name='LAST_5_CNN',
)

In [ ]:
results_df = cnn_model.data
cnn_cols = list(results_df.columns)

new_df_data = []
for index, row_data in results_df.iterrows():
    new_row = dict()
    
    for col in cnn_cols:
        new_row[col] = row_data[col]
    
    for score_index, col in enumerate(score_cols):
        new_row[col] = cnn_scores[index][score_index]
        
    new_df_data.append(new_row)

cnn_df = pd.DataFrame(new_df_data)
cnn_df.drop(columns=['round_epochs', 'batch_size', 'epochs', 'support', 'precision', 'recall', 'flatten_layer', 'kernel_size'], inplace=True)
top_5_val_cnn = cnn_df.sort_values(by=['val_accuracy'], ascending=False).head(5)
top_5_val_cnn

## Building an LSTM to predict the winner using last n fights

In [ ]:
# Get the data
X_train, X_test, y_train, y_test = collapse_n_fights()
X_train = X_train.reshape(X_train.shape[0], 5, 97)
X_test = X_test.reshape(X_test.shape[0], 5, 97)

history, model = get_lstm(X_train, y_train, X_test, y_test, test_model=True)

In [ ]:
hist = history.history
lstm_data = [hist['accuracy'][-1], hist['val_accuracy'][-1], hist['loss'][-1], hist['val_loss'][-1]]
lstm_data = lstm_data + [lstm_scores[0][0], lstm_scores[0][1], lstm_scores[0][2], lstm_scores[0][3]]
lstm_cols = ['accuracy', 'val_accuracy', 'loss', 'val_loss'] + score_cols


lstm_df = pd.DataFrame(data=[lstm_data], columns=lstm_cols)
lstm_df.sort_values(by=['val_accuracy'], ascending=False, inplace=True)
lstm_df